In [4]:
import import_ipynb
import numpy as np
import pandas as pd

#import stat_functions

In [6]:
filepath = "C:/Users/trfit/OneDrive/Documents/Hobbies/CRG/stats 2025/prepped_data/" 
file = "VL_Akron_data.csv"
df = pd.read_csv(filepath+file)


# Feature preparation

Jam Embeddings: | Jam_id | point_diff | lead | Trips | jammer_penalty_counter | blocker_penalty_counter | skater1_as_jammer |...| skatern_as_jammer | skater1_as_blocker |...| skatern_as_blocker |

curr_col = ['OP','Half','Jam',
            'LEAD', #'OP_LEAD',
            'Jammer', 'Pivot', 'Blocker_1', 'Blocker_2', 'Blocker_3',
            'Trips','OP_Trips', 
            'Jam Total', 'Game Total','OP_Jam Total', 'OP_Game Total',
            'Jammer_Box_1', 'Jammer_Box_2','Jammer_Box_3',
            'Pivot_Box_1','Pivot_Box_2','Pivot_Box_3',
            'Blocker_1_Box_1','Blocker_1_Box_2','Blocker_1_Box_3',
            'Blocker_2_Box_1','Blocker_2_Box_2','Blocker_2_Box_3',
            'Blocker_3_Box_1','Blocker_3_Box_2','Blocker_3_Box_3']

Ready: 
 - game_id: OP
 - points_for: Jam Total, Game Total
 - lead_for_us (0/1): boolean
 - our_jammer (string)
 - jam_duration: Trips
 - points_against: OP_Jam Total, OP_Game Total

Need work:
- jam_id: need to separate by game & by half
- our_blockers (list/JSON/string of 4 skater IDs): separate columns, int type

Dont have
 - penalties_our_team (int)
 - penalties_their_team (int)


box characters: 
- –  = Entered the box mid-jam.
- S = Entered between jams or continued from previous jam.
- | = Finished service.
- '+' combines - & |
- 'dollar sign' Combines + and S. Almost always paired with a - in the previous jam
- 3 = Skater injured or needed sub, sits out 3 jams.

In [3]:
# Functions

def build_encoded_matrix(df, cols, skaters):
    # This function calculates the average normalized jam +/- for each skater using matrix multiplication. This method is used because it makes 
    # it easier to remove a skater and recalculate.

    df_encoded = pd.get_dummies(df, columns=cols)

    # Combine the position boolean columns, so each skater has one boolean column, True = skater was in the jam, False = skater was not in the jam
    #cols = ['Jammer', 'Pivot', 'Blocker 1', 'Blocker 2', 'Blocker 3']
    for skater in skaters:
        cols_comb = []
        for col in cols:
            if col+"_"+str(skater) in df_encoded.columns:
                cols_comb.append(col+"_"+str(skater))
        df_encoded[str(skater)] = df_encoded[cols_comb].any(axis='columns')
    
        for col in cols_comb:
            df_encoded = df_encoded.drop(col, axis=1)

    return df_encoded

In [4]:
# Create feature df
# Jam Embeddings: | x jam_id | x point_diff | x lead | x trips | x jammer_penalty_counter | x blocker_penalty_counter | skater1_as_jammer |...| skatern_as_jammer | skater1_as_blocker |...| skatern_as_blocker |

df_features = pd.DataFrame()

# Make Jam_id
for col in ['OP', 'Half',  'Jam']:
    df[col] = df[col].astype("string")
df_features['jam_id'] = df['OP'] +"_"+ df['Half'] +"_"+ df['Jam']

# Make other features
df_features['point_diff'] = df['Jam Total'] - df['OP_Jam Total']
df_features['lead'] = df['LEAD'].replace({np.nan: False, "X": True}).infer_objects(copy=False)
df_features['trips'] = df['Trips']

# Make jammer_penalty_counter
for col in ['Jammer_Box_1', 'Jammer_Box_2', 'Jammer_Box_3']:
    df[col] = df[col].replace({np.nan: 0, '+': 1, '-': 1, '$': 1}).infer_objects(copy=False)
df_features['jammer_penalty_counter'] = df['Jammer_Box_1'] + df['Jammer_Box_2'] + df['Jammer_Box_3']

df_features['blocker_penalty_counter'] = 0
for col in ['Pivot_Box_1','Pivot_Box_2','Pivot_Box_3',
            'Blocker_1_Box_1','Blocker_1_Box_2','Blocker_1_Box_3',
            'Blocker_2_Box_1','Blocker_2_Box_2','Blocker_2_Box_3',
            'Blocker_3_Box_1','Blocker_3_Box_2','Blocker_3_Box_3']:
    df[col] = df[col].replace({np.nan: 0, '+': 1, '-': 1, '$': 1}).infer_objects(copy=False)
    df_features['blocker_penalty_counter'] += df[col]

# jammer & blocker one hot encoding
blocker_cols = ['Pivot','Blocker_1', 'Blocker_2', 'Blocker_3']
df_encoded = build_encoded_matrix(
    df[blocker_cols], blocker_cols, 
    pd.concat([df['Pivot'],df['Blocker_1'],df['Blocker_2'],df['Blocker_3']]).unique()
)

df_features = pd.concat([df_features, df_encoded], axis=1)

df_encoded = build_encoded_matrix(
    df['Jammer'], ['Jammer'], 
    df['Jammer'].unique()
)

for jammer in df['Jammer'].unique():
    df_features[str(jammer)+"_jammer"] = False
    df_features[str(jammer)+"_jammer"] = df_features[str(jammer)+"_jammer"] | df_encoded[jammer]



C:\Users\trfit\AppData\Local\Temp\ipykernel_60952\2407147693.py:13: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_features['lead'] = df['LEAD'].replace({np.nan: False, "X": True}).infer_objects(copy=False)
C:\Users\trfit\AppData\Local\Temp\ipykernel_60952\2407147693.py:18: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[col] = df[col].replace({np.nan: 0, '+': 1, '-': 1, '$': 1}).infer_objects(copy=False)
C:\Users\trfit\AppData\Local\Temp\ipykernel_60952\2407147693.py:26: FutureWarning: Downcasting behavior in `replace` is depr